This notebook handles the interaction with the Github API.
In a final step, the data must be serialized to be usable by the visualization-notebook.

Requirement: The notebook must not necessarily be able to run on JupyterLite, though this would be optimal.

**Notice:**<br>
As we found out, PyGithub does *not* provide all commit data that is needed by our tool, as it does not return the full API response when `repo.get_commits()` is run.
More specifically, it does not return the "author" field, which concerns the crucial "login" field containing the username.
Therefore, this this notebook does not work as intended.<br>
For this reason, we decided to obtain the necessary data by directly using the Github CLI.
The result of this can be found in the `commits.json` file.
This also means that the code in this notebook is deprecated and should not be used to obtain any data, as it will not work with the visualization notebook.

In [2]:
import json
from github import Github

In [3]:
# For calls that do not require authentication
# gh = Github()

# If you require authentication, use this instead. We assume that you have a "config.json" with your github_access_token in the project root directory:
with open('../config.json') as f:
	secrets = json.load(f)
access_token = secrets['github_access_token']
gh = Github(access_token)

In [6]:
repo = gh.get_repo("hpi-swt2/bookkeeper-portal-red")

Unfortunately, PYGithub does not return the full API response when running `repo.get_commits()`.
More specifically, it does not return the "author" field, which concerns the crucial "login" field containing the username.
Therefore, this this notebook does not work as intended.
It is therefore recommended to use the native Github API instead, e.g. by using the CLI tool.

In [7]:
commits = repo.get_commits()

In [8]:
# We need the _rawData (==the json response from the API) to be able to serialize it
commits = [commit.commit._rawData for commit in commits]

In [10]:
# Dump the data to a file to 'transfer' it to jupyterlite
with open('commits_DEPRECATED.json', 'w') as f:
	json.dump(commits, f, indent=4)